# Week 2, Lab 3: First-Order Logic

## Moving Beyond Propositional Logic

Propositional logic is powerful but limited. We can't express:
- "All birds can fly"
- "There exists a person who loves pizza"
- "Everyone has a mother"

**First-Order Logic (FOL)** solves this by adding:
- **Objects**: Things in the world
- **Predicates**: Properties and relations
- **Quantifiers**: "for all" (∀) and "there exists" (∃)

### What You'll Learn

- Predicates, terms, and quantifiers
- Representing relationships
- Unification algorithm
- Forward chaining with FOL
- Building a family tree reasoner

### Real-World Applications

- Database query languages (SQL)
- Semantic web (RDF, OWL)
- Natural language understanding
- Prolog programming
- Knowledge graphs

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from typing import List, Dict, Set, Tuple, Optional, Union
from copy import deepcopy

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. First-Order Logic Syntax

### Components:

1. **Constants**: Specific objects (e.g., John, Mary, 5)
2. **Variables**: Placeholders (e.g., x, y, person)
3. **Predicates**: Properties/relations (e.g., IsBird(x), Loves(x, y))
4. **Functions**: Return objects (e.g., MotherOf(x), Plus(2, 3))
5. **Quantifiers**:
   - **∀** (forall): "for all" / "every"
   - **∃** (exists): "there exists" / "some"
6. **Logical connectives**: ∧, ∨, ¬, →, ↔

### Examples:

**Propositional Logic**:
- P: "Tweety can fly"
- Q: "Tweety is a bird"

**First-Order Logic**:
- CanFly(Tweety)
- IsBird(Tweety)
- ∀x IsBird(x) → CanFly(x)
- ∃x IsBird(x) ∧ ¬CanFly(x)

In [ ]:
# Represent FOL statements as Python objects
class Term:
    """Base class for terms (constants, variables, functions)."""
    pass

class Constant(Term):
    """A constant (e.g., 'John', 'Mary')."""
    def __init__(self, name: str):
        self.name = name
    
    def __str__(self):
        return self.name
    
    def __eq__(self, other):
        return isinstance(other, Constant) and self.name == other.name
    
    def __hash__(self):
        return hash(('constant', self.name))

class Variable(Term):
    """A variable (e.g., 'x', 'y')."""
    def __init__(self, name: str):
        self.name = name
    
    def __str__(self):
        return self.name
    
    def __eq__(self, other):
        return isinstance(other, Variable) and self.name == other.name
    
    def __hash__(self):
        return hash(('variable', self.name))

class Predicate:
    """A predicate applied to terms (e.g., IsBird(Tweety))."""
    def __init__(self, name: str, *args: Term):
        self.name = name
        self.args = args
    
    def __str__(self):
        if not self.args:
            return self.name
        args_str = ", ".join(str(arg) for arg in self.args)
        return f"{self.name}({args_str})"
    
    def __eq__(self, other):
        return (isinstance(other, Predicate) and 
                self.name == other.name and 
                self.args == other.args)
    
    def __hash__(self):
        return hash(('predicate', self.name, self.args))

# Examples
print("First-Order Logic Examples:")
print("=" * 50)

# Constants
john = Constant("John")
mary = Constant("Mary")
tweety = Constant("Tweety")

# Variables
x = Variable("x")
y = Variable("y")

# Predicates
is_bird = Predicate("IsBird", tweety)
can_fly = Predicate("CanFly", tweety)
loves = Predicate("Loves", john, mary)
parent_of = Predicate("ParentOf", mary, john)

print(f"\n1. {is_bird}")
print(f"   Meaning: Tweety is a bird")

print(f"\n2. {can_fly}")
print(f"   Meaning: Tweety can fly")

print(f"\n3. {loves}")
print(f"   Meaning: John loves Mary")

print(f"\n4. {parent_of}")
print(f"   Meaning: Mary is a parent of John")

# With variables
generic_bird = Predicate("IsBird", x)
print(f"\n5. {generic_bird}")
print(f"   Meaning: x is a bird (for some x)")

## 2. Quantifiers

### Universal Quantifier (∀)
"For all" or "every"

**Example**: ∀x IsBird(x) → CanFly(x)
- "For all x, if x is a bird, then x can fly"
- "All birds can fly"

### Existential Quantifier (∃)
"There exists" or "some"

**Example**: ∃x IsBird(x) ∧ ¬CanFly(x)
- "There exists an x such that x is a bird and x cannot fly"
- "Some birds cannot fly" (penguins!)

### Negation of Quantifiers:
- ¬(∀x P(x)) ≡ ∃x ¬P(x)
- ¬(∃x P(x)) ≡ ∀x ¬P(x)

In [ ]:
# Simple knowledge base for First-Order Logic
class FOLKnowledgeBase:
    """Knowledge base for First-Order Logic facts and rules."""
    
    def __init__(self):
        self.facts = set()  # Ground facts (no variables)
        self.rules = []     # Rules with variables
    
    def add_fact(self, predicate: Predicate):
        """Add a ground fact (no variables)."""
        self.facts.add(predicate)
        print(f"Added fact: {predicate}")
    
    def add_rule(self, premises: List[Predicate], conclusion: Predicate):
        """Add a rule (premises → conclusion)."""
        self.rules.append((premises, conclusion))
        premises_str = " ∧ ".join(str(p) for p in premises)
        print(f"Added rule: {premises_str} → {conclusion}")
    
    def display(self):
        """Display the knowledge base."""
        print("\nFirst-Order Logic Knowledge Base:")
        print("=" * 60)
        
        print("\nFacts:")
        for fact in sorted(self.facts, key=str):
            print(f"  ✓ {fact}")
        
        print("\nRules:")
        for premises, conclusion in self.rules:
            premises_str = " ∧ ".join(str(p) for p in premises)
            print(f"  {premises_str} → {conclusion}")

# Build a simple KB
kb = FOLKnowledgeBase()

# Add facts
kb.add_fact(Predicate("IsBird", Constant("Tweety")))
kb.add_fact(Predicate("IsBird", Constant("Polly")))
kb.add_fact(Predicate("IsBird", Constant("Penguin")))
kb.add_fact(Predicate("IsPenguin", Constant("Penguin")))

# Add rules
# Rule 1: ∀x IsBird(x) ∧ ¬IsPenguin(x) → CanFly(x)
x = Variable("x")
kb.add_rule(
    [Predicate("IsBird", x)],
    Predicate("CanFly", x)
)

kb.display()

## 3. Unification

**Unification** finds substitutions that make expressions identical.

### Examples:

1. **Unify** IsBird(x) **with** IsBird(Tweety)
   - **Substitution**: {x/Tweety}
   - Result: IsBird(Tweety)

2. **Unify** Loves(x, y) **with** Loves(John, Mary)
   - **Substitution**: {x/John, y/Mary}
   - Result: Loves(John, Mary)

3. **Unify** Parent(x, John) **with** Parent(Mary, y)
   - **Substitution**: {x/Mary, y/John}
   - Result: Parent(Mary, John)

4. **Unify** IsBird(x) **with** IsCat(x)
   - **Cannot unify!** (different predicates)

### Unification Algorithm:
```
function UNIFY(p, q, θ):
    if θ = failure: return failure
    if p = q: return θ
    if p is variable: return UNIFY-VAR(p, q, θ)
    if q is variable: return UNIFY-VAR(q, p, θ)
    if p and q are predicates with same name:
        unify arguments recursively
    return failure
```

In [ ]:
# Substitution: mapping from variables to terms
Substitution = Dict[Variable, Term]

def apply_substitution(term: Term, subst: Substitution) -> Term:
    """Apply a substitution to a term."""
    if isinstance(term, Variable):
        return subst.get(term, term)
    elif isinstance(term, Constant):
        return term
    else:
        return term

def unify(p1: Predicate, p2: Predicate, subst: Optional[Substitution] = None) -> Optional[Substitution]:
    """
    Unify two predicates.
    
    Returns:
        Substitution that makes them identical, or None if impossible
    """
    if subst is None:
        subst = {}
    
    # Different predicate names
    if p1.name != p2.name:
        return None
    
    # Different number of arguments
    if len(p1.args) != len(p2.args):
        return None
    
    # Unify arguments pairwise
    for arg1, arg2 in zip(p1.args, p2.args):
        # Apply existing substitutions
        arg1 = apply_substitution(arg1, subst)
        arg2 = apply_substitution(arg2, subst)
        
        # If already the same, continue
        if arg1 == arg2:
            continue
        
        # Variable in arg1
        if isinstance(arg1, Variable):
            subst[arg1] = arg2
        # Variable in arg2
        elif isinstance(arg2, Variable):
            subst[arg2] = arg1
        # Both constants but different
        else:
            return None
    
    return subst

# Test unification
print("Unification Examples:")
print("=" * 60)

x = Variable("x")
y = Variable("y")
tweety = Constant("Tweety")
john = Constant("John")
mary = Constant("Mary")

# Example 1
p1 = Predicate("IsBird", x)
p2 = Predicate("IsBird", tweety)
result = unify(p1, p2)
print(f"\n1. Unify {p1} with {p2}")
if result:
    print(f"   ✓ Success! Substitution: {{{', '.join(f'{k}/{v}' for k, v in result.items())}}}")
else:
    print("   ✗ Cannot unify")

# Example 2
p3 = Predicate("Loves", x, y)
p4 = Predicate("Loves", john, mary)
result = unify(p3, p4)
print(f"\n2. Unify {p3} with {p4}")
if result:
    print(f"   ✓ Success! Substitution: {{{', '.join(f'{k}/{v}' for k, v in result.items())}}}")
else:
    print("   ✗ Cannot unify")

# Example 3 - Different predicates
p5 = Predicate("IsBird", x)
p6 = Predicate("IsCat", x)
result = unify(p5, p6)
print(f"\n3. Unify {p5} with {p6}")
if result:
    print(f"   ✓ Success! Substitution: {result}")
else:
    print("   ✗ Cannot unify (different predicates)")

# Example 4 - Both variables
p7 = Predicate("Parent", x, john)
p8 = Predicate("Parent", mary, y)
result = unify(p7, p8)
print(f"\n4. Unify {p7} with {p8}")
if result:
    print(f"   ✓ Success! Substitution: {{{', '.join(f'{k}/{v}' for k, v in result.items())}}}")
else:
    print("   ✗ Cannot unify")

## 4. Forward Chaining with Unification

Combine forward chaining with unification to reason with variables!

In [ ]:
def forward_chain_fol(kb: FOLKnowledgeBase, max_iterations: int = 10) -> Set[Predicate]:
    """
    Forward chaining with first-order logic and unification.
    """
    print("\nForward Chaining (First-Order Logic):")
    print("=" * 60)
    
    inferred = set(kb.facts)
    
    for iteration in range(max_iterations):
        new_facts = set()
        
        # Try each rule
        for premises, conclusion in kb.rules:
            # Try to match premises with known facts
            # For simplicity, we'll handle single-premise rules
            if len(premises) == 1:
                premise = premises[0]
                
                # Try to unify with each fact
                for fact in inferred:
                    subst = unify(premise, fact)
                    
                    if subst is not None:
                        # Apply substitution to conclusion
                        new_conclusion_args = tuple(
                            apply_substitution(arg, subst) 
                            for arg in conclusion.args
                        )
                        new_fact = Predicate(conclusion.name, *new_conclusion_args)
                        
                        if new_fact not in inferred:
                            new_facts.add(new_fact)
                            print(f"  Iteration {iteration + 1}: Inferred {new_fact}")
                            print(f"    From rule: {premise} → {conclusion}")
                            print(f"    Using fact: {fact}")
                            print(f"    Substitution: {{{', '.join(f'{k}/{v}' for k, v in subst.items())}}}")
        
        if not new_facts:
            print(f"\n✓ Converged after {iteration + 1} iterations")
            break
        
        inferred.update(new_facts)
    
    return inferred

# Run forward chaining
all_facts = forward_chain_fol(kb)

print("\n" + "=" * 60)
print("All Inferred Facts:")
print("=" * 60)
for fact in sorted(all_facts, key=str):
    print(f"  ✓ {fact}")

## 5. Family Tree Reasoning

Let's build a family tree and reason about relationships!

In [ ]:
# Build a family tree knowledge base
family_kb = FOLKnowledgeBase()

# People
alice = Constant("Alice")
bob = Constant("Bob")
charlie = Constant("Charlie")
diana = Constant("Diana")
eve = Constant("Eve")
frank = Constant("Frank")

# Facts: Parent relationships
family_kb.add_fact(Predicate("Parent", alice, charlie))
family_kb.add_fact(Predicate("Parent", bob, charlie))
family_kb.add_fact(Predicate("Parent", alice, diana))
family_kb.add_fact(Predicate("Parent", bob, diana))
family_kb.add_fact(Predicate("Parent", charlie, eve))
family_kb.add_fact(Predicate("Parent", charlie, frank))

# Facts: Gender
family_kb.add_fact(Predicate("Male", bob))
family_kb.add_fact(Predicate("Male", charlie))
family_kb.add_fact(Predicate("Male", frank))
family_kb.add_fact(Predicate("Female", alice))
family_kb.add_fact(Predicate("Female", diana))
family_kb.add_fact(Predicate("Female", eve))

# Rules
x, y, z = Variable("x"), Variable("y"), Variable("z")

# Rule: Parent(x, y) ∧ Male(x) → Father(x, y)
family_kb.add_rule(
    [Predicate("Parent", x, y), Predicate("Male", x)],
    Predicate("Father", x, y)
)

# Rule: Parent(x, y) ∧ Female(x) → Mother(x, y)
family_kb.add_rule(
    [Predicate("Parent", x, y), Predicate("Female", x)],
    Predicate("Mother", x, y)
)

# Rule: Parent(x, z) ∧ Parent(y, z) ∧ x≠y → Sibling(x, y)
# (Simplified - we'll just derive some relationships)

# Rule: Parent(x, y) ∧ Parent(y, z) → Grandparent(x, z)
family_kb.add_rule(
    [Predicate("Parent", x, y), Predicate("Parent", y, z)],
    Predicate("Grandparent", x, z)
)

family_kb.display()

In [ ]:
# More sophisticated forward chaining for multiple premises
def forward_chain_family(kb: FOLKnowledgeBase, max_iterations: int = 5):
    """Forward chaining that handles multiple premises."""
    print("\nForward Chaining on Family Tree:")
    print("=" * 60)
    
    inferred = set(kb.facts)
    
    for iteration in range(max_iterations):
        new_facts = set()
        
        for premises, conclusion in kb.rules:
            if len(premises) == 1:
                # Single premise (handled before)
                premise = premises[0]
                for fact in inferred:
                    subst = unify(premise, fact)
                    if subst:
                        new_args = tuple(apply_substitution(arg, subst) for arg in conclusion.args)
                        new_fact = Predicate(conclusion.name, *new_args)
                        if new_fact not in inferred:
                            new_facts.add(new_fact)
                            print(f"  Inferred: {new_fact}")
            
            elif len(premises) == 2:
                # Two premises
                p1, p2 = premises
                for fact1 in inferred:
                    subst1 = unify(p1, fact1)
                    if subst1:
                        # Apply subst1 to second premise
                        p2_instantiated_args = tuple(apply_substitution(arg, subst1) for arg in p2.args)
                        p2_instantiated = Predicate(p2.name, *p2_instantiated_args)
                        
                        for fact2 in inferred:
                            subst2 = unify(p2_instantiated, fact2, dict(subst1))
                            if subst2:
                                new_args = tuple(apply_substitution(arg, subst2) for arg in conclusion.args)
                                new_fact = Predicate(conclusion.name, *new_args)
                                if new_fact not in inferred:
                                    new_facts.add(new_fact)
                                    print(f"  Inferred: {new_fact}")
        
        if not new_facts:
            print(f"\n✓ Converged after iteration {iteration + 1}")
            break
        
        inferred.update(new_facts)
    
    return inferred

family_facts = forward_chain_family(family_kb)

print("\n" + "=" * 60)
print("Inferred Family Relationships:")
print("=" * 60)

# Organize by relationship type
relationships = {}
for fact in family_facts:
    rel_type = fact.name
    if rel_type not in relationships:
        relationships[rel_type] = []
    relationships[rel_type].append(fact)

for rel_type in sorted(relationships.keys()):
    print(f"\n{rel_type}:")
    for fact in sorted(relationships[rel_type], key=str):
        print(f"  ✓ {fact}")

In [ ]:
# Visualize the family tree
def visualize_family_tree(facts: Set[Predicate]):
    """Visualize family relationships as a graph."""
    G = nx.DiGraph()
    
    # Add edges for parent relationships
    for fact in facts:
        if fact.name == "Parent" and len(fact.args) == 2:
            parent = str(fact.args[0])
            child = str(fact.args[1])
            G.add_edge(parent, child)
    
    # Draw the tree
    plt.figure(figsize=(14, 10))
    
    # Use hierarchical layout
    pos = nx.spring_layout(G, k=2, iterations=50, seed=42)
    
    # Determine gender for coloring
    male_nodes = []
    female_nodes = []
    
    for fact in facts:
        if fact.name == "Male":
            male_nodes.append(str(fact.args[0]))
        elif fact.name == "Female":
            female_nodes.append(str(fact.args[0]))
    
    # Draw nodes by gender
    nx.draw_networkx_nodes(G, pos, nodelist=male_nodes, 
                          node_color='lightblue', node_size=3000, label='Male')
    nx.draw_networkx_nodes(G, pos, nodelist=female_nodes, 
                          node_color='lightpink', node_size=3000, label='Female')
    
    # Draw edges and labels
    nx.draw_networkx_edges(G, pos, arrows=True, arrowsize=20, 
                          width=2, alpha=0.6, edge_color='gray')
    nx.draw_networkx_labels(G, pos, font_size=12, font_weight='bold')
    
    plt.title('Family Tree\n(Arrows point from parent to child)', 
             fontsize=14, fontweight='bold')
    plt.legend()
    plt.axis('off')
    plt.tight_layout()
    plt.show()

visualize_family_tree(family_facts)

## 6. Key Takeaways

### First-Order Logic vs Propositional Logic:

| Feature | Propositional | First-Order |
|---------|--------------|-------------|
| **Objects** | No | Yes |
| **Relations** | No | Yes |
| **Variables** | No | Yes |
| **Quantifiers** | No | ∀, ∃ |
| **Expressiveness** | Limited | Rich |
| **Inference** | Simpler | More complex |

### Unification:
- **Purpose**: Match patterns with variables
- **Result**: Substitution that makes expressions identical
- **Use**: Essential for FOL inference

### Forward Chaining with FOL:
1. Start with known facts
2. Unify rule premises with facts
3. Apply substitutions to conclusions
4. Add new facts
5. Repeat until convergence

### Complexity:
- **FOL inference**: Undecidable in general
- **Datalog (safe FOL)**: Decidable
- **Horn clauses**: Polynomial time

## Next Up

In Lab 4, we'll explore:
- **Expert systems** (building practical applications)
- **SymPy** for symbolic reasoning
- **Knowledge graphs** and semantic networks
- **Real-world applications**

## Practice Exercises

1. Extend the family tree with more relationships (aunt, uncle, cousin)
2. Implement backward chaining for FOL
3. Build a knowledge base for a university domain (students, courses, professors)
4. Create a query system for the family tree
5. Implement the occurs check in unification

Excellent work! You can now represent and reason with complex relationships! 🌳🧠